In [1]:
# Core DS libraries
import numpy as np
import pandas as pd

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

import mytk # My Toolkit
import wrangle # Helper functions

- Detect if column is numerical or categorical
    - Allow manual overriding

In [2]:
df = wrangle.wrangle_potatoes()[0]

In [3]:
df.columns

Index(['product_code', 'loading', 'attribute_0', 'attribute_1', 'attribute_2',
       'attribute_3', 'measurement_0', 'measurement_1', 'measurement_2',
       'measurement_3', 'measurement_4', 'measurement_5', 'measurement_6',
       'measurement_7', 'measurement_8', 'measurement_9', 'measurement_10',
       'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14',
       'measurement_15', 'measurement_16', 'measurement_17', 'failure'],
      dtype='object')

In [7]:
l = df.select_dtypes(include=['object','category','bool']).columns.tolist()
l

['product_code', 'attribute_0', 'attribute_1']

In [13]:
def get_column_types(df, override_categorical=[], override_numerical=[]):

    cat_cols = df.select_dtypes(include=['object','category','bool']).columns.tolist()
    num_cols = df.select_dtypes(exclude=['object','category','bool']).columns.tolist()

    for val in override_categorical:
        if val in num_cols:
            num_cols.remove(val)
            cat_cols.append(val)
    for val in override_numerical:
        if val in cat_cols:
            cat_cols.remove(val)
            num_cols.append(val)

    out = {
        'cat': cat_cols,
        'num': num_cols
    }
    return out

In [18]:
gotchas = mytk.get_gotchas(df)
gotchas['possible_bools'] + gotchas['probable_categories']

['attribute_0',
 'failure',
 'product_code',
 'attribute_0',
 'attribute_1',
 'attribute_2',
 'attribute_3',
 'failure']

In [19]:
get_column_types(df, override_categorical=gotchas['possible_bools'] + gotchas['probable_categories'])

{'cat': ['product_code',
  'attribute_0',
  'attribute_1',
  'failure',
  'attribute_2',
  'attribute_3'],
 'num': ['loading',
  'measurement_0',
  'measurement_1',
  'measurement_2',
  'measurement_3',
  'measurement_4',
  'measurement_5',
  'measurement_6',
  'measurement_7',
  'measurement_8',
  'measurement_9',
  'measurement_10',
  'measurement_11',
  'measurement_12',
  'measurement_13',
  'measurement_14',
  'measurement_15',
  'measurement_16',
  'measurement_17']}